In [4]:
import gzip
from tqdm import tqdm, trange
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import folium
import pickle
from collections import defaultdict
from datetime import datetime
import matplotlib.pyplot as plt

In [5]:
places_file = "places.clean.json.gz"
users_file = "users.clean.json.gz"
reviews_file = "reviews.clean.json.gz"

user_demog_file = "user_deomograhics.pkl"
bus_demog_file = "business_deomograhics.pkl"

In [6]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)



In [ ]:
places_data = []
nrec = 0
for l in tqdm(parse(places_file)):
    # if nrec == 5:
    #     break
    places_data.append(l)
    nrec += 1

In [ ]:
users_data = []

nrec = 0
for l in tqdm(parse(users_file)):
    if nrec == 5:
        break
    users_data.append(l)
    nrec += 1

In [7]:
reviews_data = []
for l in tqdm(parse(reviews_file)):
    reviews_data.append(l)

11453845it [04:45, 40177.15it/s]


In [ ]:
with open(user_demog_file, 'rb') as f:
    user_demog = pickle.load(f)


In [11]:

with open(bus_demog_file, 'rb') as f:
    place_demog = pickle.load(f)

place_country_dict = dict()
for p, country in place_demog:
    place_country_dict[p] = country

In [9]:
modified_reviews_data = []
for d in tqdm(reviews_data):
    if remove_review(d):
        continue
    else:
        modified_reviews_data.append(d)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 11453845/11453845 [00:17<00:00, 637270.07it/s]


In [12]:
us_reviews_data = []
for d in tqdm(modified_reviews_data):
    if is_place_in_us(d, place_country_dict):
        us_reviews_data.append(d)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 10484628/10484628 [00:16<00:00, 617314.17it/s]


In [13]:
len(us_reviews_data)

4896184

In [17]:
def user_item_interactions(rev_data):
    usersPerItem = defaultdict(set)
    itemsPerUser = defaultdict(set)
    ratingsDict = defaultdict(float)
    reviewsPerUser = defaultdict(list)
    for d in tqdm(rev_data):
        u = d['gPlusUserId']
        i = d['gPlusPlaceId']
        ratingsDict[(u,i)] = d['rating']
        usersPerItem[i].add(u)
        itemsPerUser[u].add(i)
        reviewsPerUser[u].append(d)
    return usersPerItem, itemsPerUser, reviewsPerUser, ratingsDict

In [18]:
usersPerItem_us, itemsPerUser_us, reviewsPerUser_us, ratingsDict_us = user_item_interactions(us_reviews_data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4896184/4896184 [00:20<00:00, 241975.32it/s]


In [47]:
def get_updated_reviews(reviewsPerUser_us, n_review_threshold):
    us_mod_reviews_train = []
    us_mod_reviews_test = []
    for u in tqdm(reviewsPerUser_us):
        if (len(reviewsPerUser_us[u]) >= n_review_threshold):
            nelems_train = int(0.8 * len(reviewsPerUser_us[u]))
            us_mod_reviews_train.extend(reviewsPerUser_us[u][:nelems_train])
            us_mod_reviews_test.extend(reviewsPerUser_us[u][nelems_train:])
    return us_mod_reviews_train, us_mod_reviews_test

In [48]:
us_mod_reviews_train, us_mod_reviews_test = get_updated_reviews(reviewsPerUser_us, 20)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2209516/2209516 [00:00<00:00, 2598172.30it/s]


In [49]:
len(us_mod_reviews_train)

737120

In [50]:
len(us_mod_reviews_test)

194390

In [ ]:
usersPerItem_us, itemsPerUser_us, reviewsPerUser_us, ratingsDict_us = user_item_interactions(us_mod_reviews_train)

  3%|███▋                                                                                                           | 24687/737120 [00:00<00:05, 140288.05it/s]